In [2]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
from pandas.io.html import read_html

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import re
import time
import os

In [3]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [1]:
import os
for i in range(12):
    filename = '20'+str(i+10)+'-'+'20'+str(i+11)
    print(filename)
    os.mkdir("./dataset/leagues/origin/"+filename)

2010-2011
2011-2012
2012-2013
2013-2014
2014-2015
2015-2016
2016-2017
2017-2018
2018-2019


FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: './dataset/leagues/origin/2018-2019'

# 크롤링


In [4]:
tableid = {'Defensive':'statistics-table-defensive','Offensive':'statistics-table-offensive','Passing':'statistics-table-passing'}

In [7]:
url = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/6829/Stages/15151/PlayerStatistics/England-Premier-League-2017-2018'

In [43]:
def crawling(url):
    url = url
    table=['statistics-table-defensive','statistics-table-offensive','statistics-table-passing']
    url = url
    wd = webdriver.Chrome('./chromedriver.exe')
    wd.get(url)
    wd.implicitly_wait(10)


    for k in range(3):
        wd.find_element_by_link_text(list(tableid.keys())[k]).click()
        time.sleep(4)
        wd.find_element_by_link_text('All players').click()
        time.sleep(2)
        page = wd.find_element_by_link_text('last')
        total_page = int(page.get_attribute('data-page'))

        print(total_page)
        stats = read_html(wd.find_element_by_xpath('//*[@id="' +tableid[list(tableid.keys())[k]]+ '"]').get_attribute('innerHTML'))[0] #1페이지
        time.sleep(2)

        for i in range(total_page-1): #2페이지부터 늘꺼야
            findclick = wd.find_element_by_link_text('next')
            wd.implicitly_wait(10)
            findclick.click()#next 클릭
            time.sleep(1)
            playerTable = wd.find_element_by_xpath('//*[@id="' +tableid[list(tableid.keys())[k]]+ '"]')
            table_html= playerTable.get_attribute('innerHTML')
            temp = read_html(table_html)[0]
            stats = pd.concat([temp,stats], axis=0)

        if k == 0:
            defense=stats
        if k==1:
            offense=stats
        if k==2:
            passing=stats

    for columns in passing.columns:
         if columns in defense.columns:
            passing.drop([columns],axis=1,inplace=True)
    for columns in offense.columns:
         if columns in defense.columns:
            offense.drop([columns],axis=1,inplace=True)
    for columns in passing.columns:
         if columns in offense.columns:
            passing.drop([columns],axis=1,inplace=True)

    final = pd.concat([defense,offense,passing],axis=1)

    url.find('PlayerStatistics')
    filename = url[url.find('PlayerStatistics')+17:]
    final.to_csv('./dataset/leagues/origin/' + url[-9:] + '/' +filename+'.csv',encoding='utf-8-sig')

https://1xbet.whoscored.com/Regions/250/Tournaments/12/Seasons/2944/Stages/6021/PlayerStatistics/Europe-Champions-League-2011-2012

In [102]:
url = input()
crawling(url)

https://1xbet.whoscored.com/Regions/250/Tournaments/12/Seasons/2944/Stages/6021/PlayerStatistics/Europe-Champions-League-2011-2012
70
70
70


# 데이터 처리

In [183]:
final

,Player,Player.1,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Goals,Assists,SpG,KeyP,Fouled,Off,Disp,UnsTch,AvgP,PS%,Crosses,LongB,ThrB
0,"601Jérôme HergaultLorient, 36, D(LR),M(R)","Jérôme Hergault Lorient, 36, D(LR),M(R)",5(3),354,0.4,0.5,0.9,0.1,0.9,0.4,0.1,-,5.86,-,-,0.1,0.1,0.4,-,0.3,1,19.6,75.2,0.4,0.5,-
1,"602Yadaly DiabyClermont Foot, 21, Midfielder","Yadaly Diaby Clermont Foot, 21, Midfielder",0(5),64,-,0.4,0.4,-,0.4,0.2,-,-,5.85,-,-,0.4,-,0.2,-,0.2,0.4,3.6,66.7,-,-,-
2,"603Charles AbiSaint-Etienne, 22, FW","Charles Abi Saint-Etienne, 22, FW",1,46,-,-,1.0,-,-,1,-,-,5.84,-,-,-,-,-,-,-,3,6.0,50.0,-,-,-
3,"604Amadou TraoréBordeaux, 20, Forward","Amadou Traoré Bordeaux, 20, Forward",0(1),18,-,-,1.0,-,-,-,-,-,5.71,-,-,-,-,-,-,1,-,3.0,66.7,-,-,-
4,"605Maxwel CornetLyon, 25, D(L),M(LR),FW","Maxwel Cornet Lyon, 25, D(L),M(LR),FW",1,65,3,1,2.0,-,1,1,-,-,5.06,-,-,-,1,1,-,1,-,35.0,85.7,1,-,-
0,"591Romain PhilippoteauxBrest, 34, AM(LR)","Romain Philippoteaux Brest, 34, AM(LR)",0(1),23,1,-,-,-,-,-,-,-,5.95,-,-,-,-,-,1,-,2,6.0,100.0,-,-,-
1,"592Jeanuël BelocianRennes, 17, Defender","Jeanuël Belocian Rennes, 17, Defender",0(1),11,-,-,-,-,-,-,-,-,5.94,-,-,-,-,1,-,-,-,10.0,80.0,-,-,-
2,"593Johaneko Louis JeanBordeaux, 17, Defender","Johaneko Louis Jean Bordeaux, 17, Defender",0(1),11,-,-,-,-,-,-,-,-,5.94,-,-,-,-,-,-,1,1,6.0,83.3,-,1,-
3,"594Oussama TarghallineMarseille, 20, Midfielder","Oussama Targhalline Marseille, 20, Midfielder",0(2),4,-,-,1,-,-,-,-,-,5.94,-,-,-,-,-,-,-,-,3.5,100.0,-,-,-
4,"595Abdoulaye SyllaNantes, 22, Defender","Abdoulaye Sylla Nantes, 22, Defender",2,158,2,0.5,2.5,-,1.5,1,1.5,-,5.93,-,-,-,-,-,-,-,-,38.0,71.1,-,2.5,-


In [190]:
final.drop(['Player'],axis=1,inplace=True)

KeyError: "['Player'] not found in axis"

In [192]:
new = final.copy()

In [193]:
new

,Player.1,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Goals,Assists,SpG,KeyP,Fouled,Off,Disp,UnsTch,AvgP,PS%,Crosses,LongB,ThrB
0,"Jérôme Hergault Lorient, 36, D(LR),M(R)",5(3),354,0.4,0.5,0.9,0.1,0.9,0.4,0.1,-,5.86,-,-,0.1,0.1,0.4,-,0.3,1,19.6,75.2,0.4,0.5,-
1,"Yadaly Diaby Clermont Foot, 21, Midfielder",0(5),64,-,0.4,0.4,-,0.4,0.2,-,-,5.85,-,-,0.4,-,0.2,-,0.2,0.4,3.6,66.7,-,-,-
2,"Charles Abi Saint-Etienne, 22, FW",1,46,-,-,1.0,-,-,1,-,-,5.84,-,-,-,-,-,-,-,3,6.0,50.0,-,-,-
3,"Amadou Traoré Bordeaux, 20, Forward",0(1),18,-,-,1.0,-,-,-,-,-,5.71,-,-,-,-,-,-,1,-,3.0,66.7,-,-,-
4,"Maxwel Cornet Lyon, 25, D(L),M(LR),FW",1,65,3,1,2.0,-,1,1,-,-,5.06,-,-,-,1,1,-,1,-,35.0,85.7,1,-,-
0,"Romain Philippoteaux Brest, 34, AM(LR)",0(1),23,1,-,-,-,-,-,-,-,5.95,-,-,-,-,-,1,-,2,6.0,100.0,-,-,-
1,"Jeanuël Belocian Rennes, 17, Defender",0(1),11,-,-,-,-,-,-,-,-,5.94,-,-,-,-,1,-,-,-,10.0,80.0,-,-,-
2,"Johaneko Louis Jean Bordeaux, 17, Defender",0(1),11,-,-,-,-,-,-,-,-,5.94,-,-,-,-,-,-,1,1,6.0,83.3,-,1,-
3,"Oussama Targhalline Marseille, 20, Midfielder",0(2),4,-,-,1,-,-,-,-,-,5.94,-,-,-,-,-,-,-,-,3.5,100.0,-,-,-
4,"Abdoulaye Sylla Nantes, 22, Defender",2,158,2,0.5,2.5,-,1.5,1,1.5,-,5.93,-,-,-,-,-,-,-,-,38.0,71.1,-,2.5,-


In [194]:
new.reset_index(inplace=True)
new.drop(['index'],axis=1,inplace=True)
new

,Player.1,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Goals,Assists,SpG,KeyP,Fouled,Off,Disp,UnsTch,AvgP,PS%,Crosses,LongB,ThrB
0,"Jérôme Hergault Lorient, 36, D(LR),M(R)",5(3),354,0.4,0.5,0.9,0.1,0.9,0.4,0.1,-,5.86,-,-,0.1,0.1,0.4,-,0.3,1,19.6,75.2,0.4,0.5,-
1,"Yadaly Diaby Clermont Foot, 21, Midfielder",0(5),64,-,0.4,0.4,-,0.4,0.2,-,-,5.85,-,-,0.4,-,0.2,-,0.2,0.4,3.6,66.7,-,-,-
2,"Charles Abi Saint-Etienne, 22, FW",1,46,-,-,1.0,-,-,1,-,-,5.84,-,-,-,-,-,-,-,3,6.0,50.0,-,-,-
3,"Amadou Traoré Bordeaux, 20, Forward",0(1),18,-,-,1.0,-,-,-,-,-,5.71,-,-,-,-,-,-,1,-,3.0,66.7,-,-,-
4,"Maxwel Cornet Lyon, 25, D(L),M(LR),FW",1,65,3,1,2.0,-,1,1,-,-,5.06,-,-,-,1,1,-,1,-,35.0,85.7,1,-,-
5,"Romain Philippoteaux Brest, 34, AM(LR)",0(1),23,1,-,-,-,-,-,-,-,5.95,-,-,-,-,-,1,-,2,6.0,100.0,-,-,-
6,"Jeanuël Belocian Rennes, 17, Defender",0(1),11,-,-,-,-,-,-,-,-,5.94,-,-,-,-,1,-,-,-,10.0,80.0,-,-,-
7,"Johaneko Louis Jean Bordeaux, 17, Defender",0(1),11,-,-,-,-,-,-,-,-,5.94,-,-,-,-,-,-,1,1,6.0,83.3,-,1,-
8,"Oussama Targhalline Marseille, 20, Midfielder",0(2),4,-,-,1,-,-,-,-,-,5.94,-,-,-,-,-,-,-,-,3.5,100.0,-,-,-
9,"Abdoulaye Sylla Nantes, 22, Defender",2,158,2,0.5,2.5,-,1.5,1,1.5,-,5.93,-,-,-,-,-,-,-,-,38.0,71.1,-,2.5,-


In [188]:
'''new.drop([328],inplace=True)
new.reset_index(inplace=True)
new.drop(['index'], axis=1, inplace=True)
new'''

"new.drop([328],inplace=True)\nnew.reset_index(inplace=True)\nnew.drop(['index'], axis=1, inplace=True)\nnew"

In [195]:
clubs = pd.read_csv('./dataset/eplClubs.csv')
clubs

,Epl,Laliga,Bundesliga,SerieA,Ligue1
0,Leeds,Mallorca,Freiburg,Sassuolo,Lorient
1,Newcastle,Osasuna,Hoffenheim,Inter,Clermont Foot
2,Arsenal,Cadiz,Wolfsburg,Verona,Bordeaux
3,Everton,Granada,RBL,Sampdoria,Rennes
4,Brentford,Espanyol,Borussia Dortmund,Juventus,Brest
5,Aston Villa,Valencia,Bochum,Salernitana,Marseille
6,Crystal Palace,Barcelona,Frankfurt,Genoa,Monaco
7,Man Utd,Real Madrid,Mainz,Cagliari,Nantes
8,Norwich,Athletic Bilbao,Greuther Fuerth,Fiorentina,Saint-Etienne
9,Wolves,Deportivo Alaves,Hertha Berlin,Spezia,Angers


In [196]:
club_series = clubs['Ligue1'].dropna()
club_series

0           Lorient
1     Clermont Foot
2          Bordeaux
3            Rennes
4             Brest
5         Marseille
6            Monaco
7            Nantes
8     Saint-Etienne
9            Angers
10             Metz
11      Montpellier
12       Strasbourg
13           Troyes
14              PSG
15            Reims
16             Lens
17            Lille
18             Nice
19             Lyon
Name: Ligue1, dtype: object

In [197]:
position=[]
age=[]
club=[]
name=[]

for info in new['Player.1']:
    position.append(info.split()[-1])
    age.append(info.split()[-2].replace(',',''))
    for check in club_series:
        n = info.find(' '+check+',')
        if n != -1:
            club.append(check)
            name.append(info[:n])
            

In [198]:
pd.DataFrame(club)[0].nunique()

20

In [199]:
name

['Jérôme Hergault',
 'Yadaly Diaby',
 'Charles Abi',
 'Amadou Traoré',
 'Maxwel Cornet',
 'Romain Philippoteaux',
 'Jeanuël Belocian',
 'Johaneko Louis Jean',
 'Oussama Targhalline',
 'Abdoulaye Sylla',
 'Tiago Ribeiro',
 'Lucas Calodat',
 'Josh Maja',
 'Jason Mbock',
 'Ibrahim Diakité',
 'Théo Le Bris',
 'Farid El Melali',
 'Jean-Kévin Augustin',
 'Lenny Joseph',
 'Guédé Nadje',
 'Maxence Rivera',
 'Béni Makouana',
 'Habib Keita',
 'Arthur Desmas',
 'Patrick Roberts',
 'Lamine Ghezali',
 'Lebo Mothiba',
 'Mehdi Chahiri',
 'Mohamed Achi',
 'Mykola Kukharevych',
 'Rubén Pardo',
 'Sada Thioub',
 'Salim Ben Seghir',
 'Samuel Koeberle',
 'Tanguy Zoukrou',
 'Yanis Guermouche',
 'Aboubacar Sidibe',
 'Ismaël Gharbi',
 'Yanis Lhéry',
 'Wilson Isidor',
 'Edouard Michut',
 'Gautier Larsonneur',
 'Georges Mikautadze',
 'Ismaël Doukouré',
 'Jonathan Varane',
 'Oriol Busquets',
 'Mathys Saban',
 'Noah Fatar',
 'Baptiste Mouazan',
 'Matthis Abline',
 'Nathan Bitumazala',
 'Casimir Ninga',
 'Ibrahima

In [200]:
df= pd.concat([pd.DataFrame(name,columns=['Name']),pd.DataFrame(club,columns=['Club']),pd.DataFrame(age,columns=['Age']),pd.DataFrame(position,columns=['Position'])],axis=1)

In [201]:
df['Club'].unique()

array(['Lorient', 'Clermont Foot', 'Saint-Etienne', 'Bordeaux', 'Lyon',
       'Brest', 'Rennes', 'Marseille', 'Nantes', 'Monaco', 'Angers',
       'Reims', 'Metz', 'Montpellier', 'Troyes', 'Strasbourg', 'PSG',
       'Lens', 'Lille', 'Nice'], dtype=object)

In [202]:
new['Player.1']

0                Jérôme Hergault Lorient, 36, D(LR),M(R)
1             Yadaly Diaby Clermont Foot, 21, Midfielder
2                      Charles Abi Saint-Etienne, 22, FW
3                    Amadou Traoré Bordeaux, 20, Forward
4                  Maxwel Cornet Lyon, 25, D(L),M(LR),FW
5                 Romain Philippoteaux Brest, 34, AM(LR)
6                  Jeanuël Belocian Rennes, 17, Defender
7             Johaneko Louis Jean Bordeaux, 17, Defender
8          Oussama Targhalline Marseille, 20, Midfielder
9                   Abdoulaye Sylla Nantes, 22, Defender
10                    Tiago Ribeiro Monaco, 20, Defender
11             Lucas Calodat Saint-Etienne, 20, Defender
12                            Josh Maja Bordeaux, 23, FW
13                       Jason Mbock Angers, 22, Forward
14                   Ibrahim Diakité Reims, 18, Defender
15                    Théo Le Bris Lorient, 19, Defender
16                     Farid El Melali Angers, 25, AM(R)
17              Jean-Kévin Augu

In [203]:
refined = pd.concat([df,new],axis=1)
refined

,Name,Club,Age,Position,Player.1,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Goals,Assists,SpG,KeyP,Fouled,Off,Disp,UnsTch,AvgP,PS%,Crosses,LongB,ThrB
0,Jérôme Hergault,Lorient,36,"D(LR),M(R)","Jérôme Hergault Lorient, 36, D(LR),M(R)",5(3),354,0.4,0.5,0.9,0.1,0.9,0.4,0.1,-,5.86,-,-,0.1,0.1,0.4,-,0.3,1,19.6,75.2,0.4,0.5,-
1,Yadaly Diaby,Clermont Foot,21,Midfielder,"Yadaly Diaby Clermont Foot, 21, Midfielder",0(5),64,-,0.4,0.4,-,0.4,0.2,-,-,5.85,-,-,0.4,-,0.2,-,0.2,0.4,3.6,66.7,-,-,-
2,Charles Abi,Saint-Etienne,22,FW,"Charles Abi Saint-Etienne, 22, FW",1,46,-,-,1.0,-,-,1,-,-,5.84,-,-,-,-,-,-,-,3,6.0,50.0,-,-,-
3,Amadou Traoré,Bordeaux,20,Forward,"Amadou Traoré Bordeaux, 20, Forward",0(1),18,-,-,1.0,-,-,-,-,-,5.71,-,-,-,-,-,-,1,-,3.0,66.7,-,-,-
4,Maxwel Cornet,Lyon,25,"D(L),M(LR),FW","Maxwel Cornet Lyon, 25, D(L),M(LR),FW",1,65,3,1,2.0,-,1,1,-,-,5.06,-,-,-,1,1,-,1,-,35.0,85.7,1,-,-
5,Romain Philippoteaux,Brest,34,AM(LR),"Romain Philippoteaux Brest, 34, AM(LR)",0(1),23,1,-,-,-,-,-,-,-,5.95,-,-,-,-,-,1,-,2,6.0,100.0,-,-,-
6,Jeanuël Belocian,Rennes,17,Defender,"Jeanuël Belocian Rennes, 17, Defender",0(1),11,-,-,-,-,-,-,-,-,5.94,-,-,-,-,1,-,-,-,10.0,80.0,-,-,-
7,Johaneko Louis Jean,Bordeaux,17,Defender,"Johaneko Louis Jean Bordeaux, 17, Defender",0(1),11,-,-,-,-,-,-,-,-,5.94,-,-,-,-,-,-,1,1,6.0,83.3,-,1,-
8,Oussama Targhalline,Marseille,20,Midfielder,"Oussama Targhalline Marseille, 20, Midfielder",0(2),4,-,-,1,-,-,-,-,-,5.94,-,-,-,-,-,-,-,-,3.5,100.0,-,-,-
9,Abdoulaye Sylla,Nantes,22,Defender,"Abdoulaye Sylla Nantes, 22, Defender",2,158,2,0.5,2.5,-,1.5,1,1.5,-,5.93,-,-,-,-,-,-,-,-,38.0,71.1,-,2.5,-


In [204]:
refined.drop(['Player.1'],axis=1,inplace=True)

In [205]:
refined

,Name,Club,Age,Position,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Goals,Assists,SpG,KeyP,Fouled,Off,Disp,UnsTch,AvgP,PS%,Crosses,LongB,ThrB
0,Jérôme Hergault,Lorient,36,"D(LR),M(R)",5(3),354,0.4,0.5,0.9,0.1,0.9,0.4,0.1,-,5.86,-,-,0.1,0.1,0.4,-,0.3,1,19.6,75.2,0.4,0.5,-
1,Yadaly Diaby,Clermont Foot,21,Midfielder,0(5),64,-,0.4,0.4,-,0.4,0.2,-,-,5.85,-,-,0.4,-,0.2,-,0.2,0.4,3.6,66.7,-,-,-
2,Charles Abi,Saint-Etienne,22,FW,1,46,-,-,1.0,-,-,1,-,-,5.84,-,-,-,-,-,-,-,3,6.0,50.0,-,-,-
3,Amadou Traoré,Bordeaux,20,Forward,0(1),18,-,-,1.0,-,-,-,-,-,5.71,-,-,-,-,-,-,1,-,3.0,66.7,-,-,-
4,Maxwel Cornet,Lyon,25,"D(L),M(LR),FW",1,65,3,1,2.0,-,1,1,-,-,5.06,-,-,-,1,1,-,1,-,35.0,85.7,1,-,-
5,Romain Philippoteaux,Brest,34,AM(LR),0(1),23,1,-,-,-,-,-,-,-,5.95,-,-,-,-,-,1,-,2,6.0,100.0,-,-,-
6,Jeanuël Belocian,Rennes,17,Defender,0(1),11,-,-,-,-,-,-,-,-,5.94,-,-,-,-,1,-,-,-,10.0,80.0,-,-,-
7,Johaneko Louis Jean,Bordeaux,17,Defender,0(1),11,-,-,-,-,-,-,-,-,5.94,-,-,-,-,-,-,1,1,6.0,83.3,-,1,-
8,Oussama Targhalline,Marseille,20,Midfielder,0(2),4,-,-,1,-,-,-,-,-,5.94,-,-,-,-,-,-,-,-,3.5,100.0,-,-,-
9,Abdoulaye Sylla,Nantes,22,Defender,2,158,2,0.5,2.5,-,1.5,1,1.5,-,5.93,-,-,-,-,-,-,-,-,38.0,71.1,-,2.5,-


In [206]:
refined.to_csv('./dataset/refined'+filename+'.csv')